In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import pandas as pd
import scipy.stats as stats
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops, hog
from skimage.color import rgb2gray
from skimage.measure import shannon_entropy
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils.class_weight import compute_class_weight

# ============ IMAGE ENHANCEMENT FUNCTION ============
def enhance_image(image):
    """Applies CLAHE to enhance image contrast."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))  # Increased clipLimit
    return clahe.apply(gray)

# ============ EXTRACT TEXTURE FEATURES FUNCTION ============
def extract_texture_features(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))

    gray = rgb2gray(image)  
    gray = (gray * 255).astype(np.uint8)  # Convert to uint8

    # Enhancement
    enhanced_gray = enhance_image(image)

    # Gabor Features
    gabor_kernels = [cv2.getGaborKernel((5, 5), sigma, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
                     for sigma in [1, 3] for theta in [0, np.pi/4, np.pi/2, 3*np.pi/4]]
    gabor_features = [np.mean(cv2.filter2D(enhanced_gray, cv2.CV_8UC3, k)) for k in gabor_kernels]

    # GLCM Features
    glcm = graycomatrix(enhanced_gray, distances=[1, 2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], symmetric=True, normed=True)
    glcm_features = [graycoprops(glcm, prop).mean() for prop in ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']]

    # LBP Features
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(11), density=True)

    # HOG Features
    hog_features = hog(gray, orientations=6, pixels_per_cell=(8, 8), cells_per_block=(1, 1), feature_vector=True)

    # Statistical Features
    ent = shannon_entropy(enhanced_gray)
    mean, variance, skewness, kurtosis = np.mean(enhanced_gray), np.var(enhanced_gray), stats.skew(enhanced_gray.flatten()), stats.kurtosis(enhanced_gray.flatten())

    # SIFT Features
    sift = cv2.SIFT_create()
    keypoints, _ = sift.detectAndCompute(enhanced_gray, None)
    sift_count = len(keypoints)

    # Combine all features
    combined_features = np.hstack((glcm_features, lbp_hist, hog_features, gabor_features, [ent, mean, variance, skewness, kurtosis, sift_count]))
    return combined_features

# ============ LOAD DATASET FUNCTION ============
def load_dataset(split):
    tree_images = tf.io.gfile.glob(f"dataset/{split}/tree/*")
    non_tree_images = tf.io.gfile.glob(f"dataset/{split}/non_tree/*")

    image_paths = tree_images + non_tree_images
    labels = [1] * len(tree_images) + [0] * len(non_tree_images)

    # Extract features
    texture_features = [extract_texture_features(img) for img in image_paths]

    return np.array(image_paths), np.array(labels, dtype=np.float32), np.array(texture_features, dtype=np.float32)

# ============ CUSTOM DATA GENERATOR ============
class CustomDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_generator, manual_features, batch_size):
        self.image_generator = image_generator
        self.manual_features = manual_features
        self.batch_size = batch_size

    def __len__(self):
        return len(self.image_generator)

    def __getitem__(self, index):
        image_batch, label_batch = self.image_generator[index]

        start_idx = index * self.batch_size
        end_idx = (index + 1) * self.batch_size
        manual_batch = self.manual_features[start_idx:end_idx]

        manual_batch_tensor = tf.convert_to_tensor(manual_batch, dtype=tf.float32)
        image_batch_tensor = tf.convert_to_tensor(image_batch, dtype=tf.float32)

        return (image_batch_tensor, manual_batch_tensor), label_batch

# ============ BUILD MODEL ============
def build_model():
    image_input = layers.Input(shape=(128, 128, 3), name="image_input")
    base_cnn = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_tensor=image_input)

    for layer in base_cnn.layers[-10:]:
        layer.trainable = True

    cnn_output = layers.GlobalAveragePooling2D()(base_cnn.output)

    feature_input = layers.Input(shape=(1566,), name="feature_input")  
    feature_branch = layers.Dense(128, activation='relu')(feature_input)
    feature_branch = layers.Dense(64, activation='relu')(feature_branch)

    merged = layers.Concatenate()([cnn_output, feature_branch])
    attention = layers.Dense(256, activation='relu')(merged)
    output = layers.Dense(1, activation='sigmoid')(attention)

    model = models.Model(inputs=[image_input, feature_input], outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# ============ LOAD DATA ============
train_paths, train_labels, train_features = load_dataset('train')
val_paths, val_labels, val_features = load_dataset('val')

train_labels = train_labels.astype(str)
val_labels = val_labels.astype(str)

train_labels = train_labels.astype(float).astype(int)


train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
).flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": train_paths, "class": train_labels.astype(str)}),  # Convert labels to string
    x_col="filename", y_col="class", target_size=(128, 128), batch_size=32, class_mode='binary'
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": val_paths, "class": val_labels.astype(str)}),  # Convert labels to string
    x_col="filename", y_col="class", target_size=(128, 128), batch_size=32, class_mode='binary'
)

dataset_train = CustomDataGenerator(train_gen, train_features, batch_size=32)
dataset_val = CustomDataGenerator(val_gen, val_features, batch_size=32)

# ============ TRAINING ============
model = build_model()

class_weights = compute_class_weight('balanced', classes=np.array([0, 1]), y=train_labels)
class_weight_dict = {i: class_weights[i] for i in range(2)}

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6, verbose=1)

history = model.fit(
    dataset_train,
    validation_data=dataset_val,
    epochs=8,
    class_weight=class_weight_dict,
    callbacks=[reduce_lr]
)

model.save("tree_detection_model.h5")  # Saves the model as a .h5 file
